In [1]:
import os

if "notebooks" in os.getcwd():
    os.chdir("..")

In [2]:
import glob
import pandas as pd

In [ ]:
# df_main = pd.DataFrame()

# for input_path in sorted(glob.glob("data/inputs/*_response.csv")):
#     df = pd.read_csv(input_path)
#     df_main = pd.concat([df_main, df], axis=0, ignore_index=True)
#     print(df.shape, df_main.shape)

(100000, 8) (100000, 8)
(100000, 8) (200000, 8)
(100000, 8) (300000, 8)
(100000, 8) (400000, 8)
(100000, 8) (500000, 8)


# Last Batch from 001

In [ ]:
# take all urls from 001 file, read all links in batch 1 and batch 2, concat them, compare with 001 and get common urls

# now filter 001 file by removing common urls

### Inputs

In [17]:
df_input = pd.read_csv('data/inputs/100k_response - 100k_response.csv')
print(df_input.shape)

# take only source_url
df_input = df_input[["source_url"]]

# take only urls with event
df_input = df_input[df_input['source_url'].str.contains(r'^https://dice\.fm/event/.*$', regex=True)]
print(df_input.shape)

# remove everything after ? in source_url
df_input['source_url'] = df_input['source_url'].str.replace(r'\?.*$', '', regex=True)

# drop duplicates
df_input = df_input.drop_duplicates()
print(df_input.shape)

print(df_input.columns)

(100000, 8)
(46333, 1)
(41535, 1)
Index(['source_url'], dtype='object')


### outputs

In [19]:
df_output = pd.DataFrame()

df_output_1 = pd.read_excel("data/outputs/dice_events_DICE Event Links 1st batch - Sheet1_20241217_171759.xlsx")
print(df_output_1.shape)
df_output = pd.concat([df_output, df_output_1], axis=0, ignore_index=True)

df_output_2 = pd.read_excel("data/outputs/dice_events_Dice Batch 3_20241218_093139.xlsx")
print(df_output_1.shape)
df_output = pd.concat([df_output, df_output_2], axis=0, ignore_index=True)

df_output = df_output[["url"]]
print(df_output.shape)
df_output = df_output.drop_duplicates()
print(df_output.shape)

print(df_output.columns)

(165579, 15)
(165579, 15)
(304032, 1)
(304031, 1)
Index(['url'], dtype='object')


In [20]:
common_urls = set(df_input['source_url']).intersection(set(df_output['url']))
df_new_batch = df_input[~df_input['source_url'].isin(common_urls)]
print(df_new_batch.shape)
print(df_new_batch.columns)

(2, 1)
Index(['source_url'], dtype='object')


### read latest batch on inputs

In [21]:
df_latest_full_batch = pd.DataFrame()

for input_path in sorted(glob.glob("data/inputs/*dice.fm_event*.csv")):
    print(input_path)
    df = pd.read_csv(input_path, low_memory=False)
    df.columns = ['source_url', 'source_title', 'response_code', 'backlinks_num',
       'domains_num', 'last_seen', 'external_num', 'internal_num']
    print(df.shape)
    print(df.columns)
    df_latest_full_batch = pd.concat([df_latest_full_batch, df], axis=0, ignore_index=True)
    print(df.shape, df_latest_full_batch.shape)

df_latest_full_batch = df_latest_full_batch[["source_url"]]

data/inputs/dice.fm_event - dice.fm_event.csv
(270356, 8)
Index(['source_url', 'source_title', 'response_code', 'backlinks_num',
       'domains_num', 'last_seen', 'external_num', 'internal_num'],
      dtype='object')
(270356, 8) (270356, 8)
data/inputs/dice.fm_event-csv2 - dice.fm_event-csv2.csv
(149648, 8)
Index(['source_url', 'source_title', 'response_code', 'backlinks_num',
       'domains_num', 'last_seen', 'external_num', 'internal_num'],
      dtype='object')
(149648, 8) (420004, 8)


In [22]:
df_latest_full_batch = pd.concat([df_latest_full_batch, df_new_batch], axis=0, ignore_index=True)
print(df_latest_full_batch.shape)
print(df_latest_full_batch.columns)

(420006, 1)
Index(['source_url'], dtype='object')


In [26]:
common_urls = set(df_latest_full_batch['source_url']).intersection(set(df_output['url']))
df_last_batch = df_latest_full_batch[~df_latest_full_batch['source_url'].isin(common_urls)]
df_last_batch = df_last_batch.drop_duplicates()
print(df_last_batch.shape)
print(df_last_batch.columns)

(123062, 1)
Index(['source_url'], dtype='object')


In [28]:
df_last_batch.to_csv("data/inputs/Dice_Batch_4.csv")